In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir('../')

In [2]:
import transtab

# set random seed
transtab.random_seed(42)

In [4]:
import numpy as np
import pandas as pd

In [ ]:
# load a dataset and start vanilla supervised training
# !pip install openml
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols = transtab.load_data(['credit-g', 'credit-approval'])


########################################


ImportError: OpenML is required for this functionality. Please install it with: pip install openml

In [3]:
trainset_reg = [(trainset[0][0], pd.Series(np.random.randn(trainset[0][0].shape[0]))), (trainset[1][0], pd.Series(np.random.randn(trainset[1][0].shape[0])))]

NameError: name 'trainset' is not defined

In [38]:
valset_reg = [(valset[0][0], pd.Series(np.random.randn(valset[0][0].shape[0]))), (valset[1][0], pd.Series(np.random.randn(valset[1][0].shape[0])))]
testset_reg = [(testset[0][0], pd.Series(np.random.randn(testset[0][0].shape[0]))), (testset[1][0], pd.Series(np.random.randn(testset[1][0].shape[0])))]

In [39]:

# build transtab classifier model
model = transtab.build_regressor(cat_cols, num_cols, bin_cols, device='cpu')

# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'val_loss',
    'eval_less_is_better':True,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':1e-4,
    'weight_decay':1e-4,
    }
transtab.train(model, trainset_reg[0], valset_reg[0], **training_arguments)

# save model
model.save('./ckpt/pretrained')

Epoch:   2%|▏         | 1/50 [00:01<01:33,  1.91s/it]

epoch: 0, test val_loss: 1.372377
epoch: 0, train loss: 6.7940, lr: 0.000100, spent: 1.9 secs


Epoch:   4%|▍         | 2/50 [00:03<01:22,  1.72s/it]

epoch: 1, test val_loss: 1.184756
epoch: 1, train loss: 6.0480, lr: 0.000100, spent: 3.5 secs


Epoch:   6%|▌         | 3/50 [00:05<01:18,  1.66s/it]

...

epoch: 4, test val_loss: 1.198663
EarlyStopping counter: 3 out of 5
epoch: 4, train loss: 5.9642, lr: 0.000100, spent: 8.3 secs


Epoch:  12%|█▏        | 6/50 [00:09<01:10,  1.61s/it]

epoch: 5, test val_loss: 1.205427
EarlyStopping counter: 4 out of 5
epoch: 5, train loss: 5.8004, lr: 0.000100, spent: 9.9 secs


Epoch:  12%|█▏        | 6/50 [00:11<01:24,  1.92s/it]
2024-03-08 16:32:55.367 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2024-03-08 16:32:55.378 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint
2024-03-08 16:32:55.471 | INFO     | transtab.trainer:train:141 - training complete, cost 11.6 secs.


epoch: 6, test val_loss: 1.198403
EarlyStopping counter: 5 out of 5
early stopped


In [40]:
# now let's use another data and try to leverage the pretrained model for finetuning
# here we have loaded the required data `credit-approval` before, no need to load again.

# load the pretrained model
model.load('./ckpt/pretrained')

# update model's categorical/numerical/binary column dict
# need to specify the number of classes if the new dataset has different # of classes from the 
# pretrained one.
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols, 'num_class':2})

2024-03-08 16:33:11.448 | INFO     | transtab.modeling_transtab:load:787 - missing keys: []
2024-03-08 16:33:11.448 | INFO     | transtab.modeling_transtab:load:788 - unexpected keys: []
2024-03-08 16:33:11.449 | INFO     | transtab.modeling_transtab:load:789 - load model from ./ckpt/pretrained


2024-03-08 16:33:11.468 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from ./ckpt/pretrained/extractor/extractor.json
2024-03-08 16:33:11.470 | INFO     | transtab.modeling_transtab:_adapt_to_new_num_class:886 - Build a new classifier with num 2 classes outputs, need further finetune to work.


In [44]:
# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'mse',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':2e-4,
    }

transtab.train(model, trainset_reg[1], valset_reg[1], **training_arguments)


Epoch:   2%|▏         | 1/50 [00:00<00:37,  1.32it/s]

epoch: 0, test mse: 0.814842
epoch: 0, train loss: 2.9249, lr: 0.000200, spent: 0.8 secs


Epoch:   4%|▍         | 2/50 [00:01<00:31,  1.54it/s]

epoch: 1, test mse: 0.803411
EarlyStopping counter: 1 out of 5
epoch: 1, train loss: 0.1003, lr: 0.000200, spent: 1.3 secs


Epoch:   6%|▌         | 3/50 [00:01<00:29,  1.57it/s]

...

epoch: 3, test mse: 0.802881
EarlyStopping counter: 3 out of 5
epoch: 3, train loss: -0.3803, lr: 0.000200, spent: 2.6 secs


Epoch:  10%|█         | 5/50 [00:03<00:28,  1.60it/s]

epoch: 4, test mse: 0.802826
EarlyStopping counter: 4 out of 5
epoch: 4, train loss: -0.2638, lr: 0.000200, spent: 3.2 secs


Epoch:  10%|█         | 5/50 [00:03<00:33,  1.34it/s]
2024-03-08 16:37:52.614 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2024-03-08 16:37:52.621 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint
2024-03-08 16:37:52.718 | INFO     | transtab.trainer:train:141 - training complete, cost 3.9 secs.


epoch: 5, test mse: 0.802803
EarlyStopping counter: 5 out of 5
early stopped


In [46]:
# evaluation
x_test, y_test = testset_reg[1]
ypred = transtab.predict(model, x_test, y_test)
transtab.evaluate(ypred, y_test, metric='mse')

from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, ypred))

0.9819256995837686
